In [84]:
import datetime
import RunStrategy
import atomic as atom
import definitions as defs
from pathlib import Path
import pandas as pd
import strategies
import reporting as rep
import warnings
import GetConfigs
import time
import DefaultConfigs as defcon
import OptimizedConfigs as opcon
import numpy as np

warnings.filterwarnings("ignore")

Root = "../"

Banknifty_Path = Root + "NIFTYOptionsData/OptionsData/Banknifty/"
Nifty_Path = Root + "NIFTYOptionsData/OptionsData/Nifty/"

In [89]:
# INDOLS Config with Bandit: Bank Nifty 

config = defcon.ind_straddle_BN_OLS
print(config)

start_date = datetime.date(2022, 1, 1)
end_date = datetime.date(2022, 12, 31)

startTimes = [datetime.time(9, 16), datetime.time(9, 30), datetime.time(10, 0),datetime.time(10, 30), datetime.time(11, 0)]
endTimes = [datetime.time(14, 0), datetime.time(14, 30), datetime.time(15, 0), datetime.time(15, 15)]
pairTimes = []
for s in startTimes:
  for e in endTimes:
    pairTimes.append([s, e, [], 0])

delta = datetime.timedelta(days=1)
trade = pd.DataFrame()
trades = pd.DataFrame()
PNLTrackerRunning = pd.DataFrame()
tic = time.time()
origDelta = config["Delta"]
while start_date <= end_date:
  print(start_date)
  if start_date.weekday() == defs.THU:
    config["Delta"] = config["DeltaThu"]
  else:
    config["Delta"] = origDelta
  date_string = start_date.strftime("%Y/Data%Y%m%d.csv")
  BNPath = Banknifty_Path + date_string
  NPath = Nifty_Path + date_string
  my_fileN = Path(NPath)
  my_fileBN = Path(BNPath)
  runToday = ((not config["OnlyThu"]) or (start_date.weekday() == defs.THU)) and (config["RunFriday"] or start_date.weekday() != defs.FRI)
  for p in pairTimes:
    config["EnterTime"] = p[0]
    config["ExitTime"] = p[1]
    (generalconfig, positionconfig) = GetConfigs.GetINDStranglesConfig(config)
    if my_fileN.exists() and my_fileBN.exists() and runToday:
      masterdfN = atom.LoadDF(NPath)
      masterdfBN = atom.LoadDF(BNPath)
      if (generalconfig["symbol"] == defs.BN):
        (trade, PNLTracker, PNLTrackerSumm) = strategies.IntraDayStrategy(masterdfBN, generalconfig, positionconfig)
      elif (generalconfig["symbol"] == defs.N):
        (trade, PNLTracker, PNLTrackerSumm) = strategies.IntraDayStrategy(masterdfN, generalconfig, positionconfig)
      p[3] = p[3] + PNLTrackerSumm["FinalPNL"]
      p[2].append(PNLTrackerSumm["FinalPNL"])
  start_date += delta

{'SquareOffSL': 1, 'SquareOffTG': 1, 'symbol': 'BANKNIFTY', 'ReEntrySL': 0, 'ReEntryTG': 0, 'REEvery': 14, 'TrailSL': 0, 'MaxReEnterCounterSL': 3, 'MaxReEnterCounterTG': 6, 'SLtoCost': 1, 'SL': 1, 'Target': 0, 'SLPc': 15, 'TargetPc': 75, 'Delta': 0, 'DeltaThu': 0, 'OnlyThu': False, 'EnterTime': datetime.time(11, 0), 'ExitTime': datetime.time(15, 15), 'action': -1, 'Limits': 0, 'PNLLimit': -3000, 'DrawdownLimit': -4000, 'RunFriday': True}
2022-01-01
2022-01-02
2022-01-03
2022-01-04
2022-01-05
2022-01-06
2022-01-07
2022-01-08
2022-01-09
2022-01-10
2022-01-11
2022-01-12
2022-01-13
2022-01-14
2022-01-15
2022-01-16
2022-01-17
2022-01-18
2022-01-19
2022-01-20
2022-01-21
2022-01-22
2022-01-23
2022-01-24
2022-01-25
2022-01-26
2022-01-27
2022-01-28
2022-01-29
2022-01-30
2022-01-31
2022-02-01
2022-02-02
2022-02-03
2022-02-04
2022-02-05
2022-02-06
2022-02-07
2022-02-08
2022-02-09
2022-02-10
2022-02-11
2022-02-12
2022-02-13
2022-02-14
2022-02-15
2022-02-16
2022-02-17
2022-02-18
2022-02-19
2022-02-

In [100]:
runningPNL = np.zeros(len(pairTimes))
stratPNL = 0
daysLookBack = 40
numDays = len(pairTimes[0][2])
for day in range(numDays):
    maxPNL = runningPNL[0]
    currval = 0
    bestidx = [pairTimes[0][0], pairTimes[0][1]]
    for i in range(len(pairTimes)):
        if (runningPNL[i] >= maxPNL):
            currval = pairTimes[i][2][day]
            bestidx = [pairTimes[i][0], pairTimes[i][1]]
            maxPNL = runningPNL[i]            
        runningPNL[i] = runningPNL[i] + pairTimes[i][2][day]
        if (day >= daysLookBack):
            runningPNL[i] = runningPNL[i] - pairTimes[i][2][day - daysLookBack]
    stratPNL = stratPNL + currval
    print(bestidx)
print(stratPNL) 
print(runningPNL)

[datetime.time(11, 0), datetime.time(15, 15)]
[datetime.time(9, 30), datetime.time(15, 15)]
[datetime.time(10, 30), datetime.time(14, 0)]
[datetime.time(11, 0), datetime.time(14, 30)]
[datetime.time(11, 0), datetime.time(14, 30)]
[datetime.time(11, 0), datetime.time(14, 30)]
[datetime.time(11, 0), datetime.time(14, 30)]
[datetime.time(11, 0), datetime.time(14, 30)]
[datetime.time(11, 0), datetime.time(14, 30)]
[datetime.time(11, 0), datetime.time(14, 30)]
[datetime.time(11, 0), datetime.time(14, 30)]
[datetime.time(11, 0), datetime.time(14, 30)]
[datetime.time(11, 0), datetime.time(14, 30)]
[datetime.time(11, 0), datetime.time(14, 30)]
[datetime.time(11, 0), datetime.time(14, 30)]
[datetime.time(11, 0), datetime.time(14, 30)]
[datetime.time(10, 0), datetime.time(14, 30)]
[datetime.time(11, 0), datetime.time(14, 30)]
[datetime.time(11, 0), datetime.time(14, 30)]
[datetime.time(11, 0), datetime.time(14, 30)]
[datetime.time(11, 0), datetime.time(14, 30)]
[datetime.time(11, 0), datetime.ti

In [91]:
for p in pairTimes:
    print(p[0], p[1], p[3])

09:16:00 14:00:00 34138.61319375057
09:16:00 14:30:00 44895.288006250536
09:16:00 15:00:00 42893.34322500056
09:16:00 15:15:00 30969.917193750538
09:30:00 14:00:00 67458.44928593803
09:30:00 14:30:00 83476.06667968804
09:30:00 15:00:00 88664.02914218804
09:30:00 15:15:00 79433.09720468806
10:00:00 14:00:00 30976.452717187985
10:00:00 14:30:00 42385.22720937545
10:00:00 15:00:00 40299.527834375476
10:00:00 15:15:00 23096.389490625526
10:30:00 14:00:00 42757.65336093796
10:30:00 14:30:00 55934.40102968797
10:30:00 15:00:00 32645.668670312967
10:30:00 15:15:00 16186.98376406298
11:00:00 14:00:00 35132.50680781294
11:00:00 14:30:00 49122.96816250044
11:00:00 15:00:00 32230.511068750493
11:00:00 15:15:00 29402.683256250504
